In [ ]:
import numpy as np
from pymatgen.core import Structure
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import json

In [ ]:
# 对DFT直接拟合

In [ ]:
with open("/root/home/jupyter/DFT/bandgap/bandgap_json/DFT_bandgapTrain1.json", "r") as f:
    data_train = json.load(f)

In [ ]:
structures = []
DFT_bandgaps = []

for item in data_train:
    structure = Structure.from_dict(item['structure'])
    gap = item['bandgap']
    structures.append(structure)
    DFT_bandgaps.append(gap)

In [ ]:
#5是最低截断半径，用以判断哪两个原子之间有键连接

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 设置参数列表
nfeat_bond_list = [10, 15, 20, 25, 30]
r_cutoff_list = [6, 7, 8]

# 初始化结果记录
results = []

# 循环训练模型
for nfeat_bond in nfeat_bond_list:
    for r_cutoff in r_cutoff_list:
        # 设置参数
        gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
        gaussian_width = 0.5
        graph_converter = CrystalGraph(cutoff=r_cutoff)
        
        # 初始化模型
        model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)
        
        # 训练模型
        model.train(structures,DFT_bandgaps, epochs=100)
        
        # 预测结果
        predictions = []
        for structure in structures:
            prediction = model.predict_structure(structure)
            predictions.append(prediction[0])
        
        # 计算评估指标
        mse = mean_squared_error(DFT_bandgaps, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(DFT_bandgaps, predictions)
        r2 = r2_score(DFT_bandgaps, predictions)
        
        # 记录结果
        results.append({
            'nfeat_bond': nfeat_bond,
            'r_cutoff': r_cutoff,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })



In [ ]:
# 打印结果
for result in results:
    print(f"nfeat_bond: {result['nfeat_bond']}, r_cutoff: {result['r_cutoff']}, MSE: {result['mse']}, RMSE: {result['rmse']}, MAE: {result['mae']}, R2: {result['r2']}")


In [ ]:
def get_best_r2(results):
    # 找到R2最大的结果
    best_result = max(results, key=lambda result: result['r2'])
    return best_result

def get_best_rmse(results):
    # 计算每个结果的RMSE
    for result in results:
        result['rmse'] = np.sqrt(result['mse'])
    
    # 找到RMSE最小的结果
    best_result = min(results, key=lambda result: result['rmse'])
    return best_result


In [ ]:
best_r2_result = get_best_r2(results)
print(f"Best R2: {best_r2_result['r2']}, nfeat_bond: {best_r2_result['nfeat_bond']}, r_cutoff: {best_r2_result['r_cutoff']}")

best_rmse_result = get_best_rmse(results)
print(f"Best RMSE: {best_rmse_result['rmse']}, nfeat_bond: {best_rmse_result['nfeat_bond']}, r_cutoff: {best_rmse_result['r_cutoff']}")


In [ ]:
def get_best_mae(results):
    # 计算每个结果的MAE
    for result in results:
        result['mae'] = mean_absolute_error(gaps, predictions)
    
    # 找到MAE最小的结果
    best_result = min(results, key=lambda result: result['mae'])
    return best_result

In [ ]:
best_mae_result = get_best_mae(results)
print(f"Best MAE: {best_mae_result['mae']}, nfeat_bond: {best_mae_result['nfeat_bond']}, r_cutoff: {best_mae_result['r_cutoff']}")


In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 设置参数列表
nfeat_bond_list2 = [10]
r_cutoff_list2 = [10,12]#越高，计算时间越长，更多的边，键长信息将被处理

# 初始化结果记录
results2 = []

# 循环训练模型
for nfeat_bond in nfeat_bond_list2:
    for r_cutoff in r_cutoff_list2:
        # 设置参数
        gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
        gaussian_width = 0.5
        graph_converter = CrystalGraph(cutoff=r_cutoff)
        
        # 初始化模型
        model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)
        
        # 训练模型
        model.train(structures, gaps, epochs=100)
        
        # 预测结果
        predictions = []
        for structure in structures:
            prediction = model.predict_structure(structure)
            predictions.append(prediction[0])
        
        # 计算评估指标
        mse = mean_squared_error(gaps, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(gaps, predictions)
        r2 = r2_score(gaps, predictions)
        
        # 记录结果
        results2.append({
            'nfeat_bond': nfeat_bond,
            'r_cutoff': r_cutoff,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        })

In [ ]:
# 打印结果
for result in results2:
    print(f"nfeat_bond: {result['nfeat_bond']}, r_cutoff: {result['r_cutoff']}, MSE: {result['mse']}, RMSE: {result['rmse']}, MAE: {result['mae']}, R2: {result['r2']}")
